# Note:
This codes about data preparation part and sample code. We can a lot of change on the code while working our local computers but also these codes give information about our implementation details.

## Extract Frame From Videos

In [ ]:
import cv2
import os
import glob
videos = glob.glob("C:/BitirmeProjesi/Normal/*.mp4")


for i in videos:
    os.mkdir("C:/BitirmeProjesi/NormalFrames/video"+(str(i).split("\\"))[1].split(".")[0]) 
    vidcap = cv2.VideoCapture(i)
    success,image = vidcap.read()
    count = 1
    writer = 1
    while success:
        if(count!=1 and count%2==1):
            
            cv2.imwrite("C:/BitirmeProjesi/NormalFrames/video"+(str(i).split("\\"))[1].split(".")[0]+"/frame"+str(writer)+".jpg", image)
            writer +=1    
        success,image = vidcap.read()
        print('Read a new frame: ', success)
        count += 1

## Create Labels From File

In [ ]:
import glob
import numpy as np
from numpy import savez_compressed
import os



labelFilePath = "Labels File Path Here"
labels = np.arange(60000)
labels.fill(0)
k = 1
for i in range(1500):
    data1 = np.load(labelFilePath+"label"+str(k)+".npz",allow_pickle = True)
    array1 = data1[data1.files[0]]
    for j in range(4,24):
        labels = np.concatenate((labels,np.array([array1[j]])),axis = 0)
    k+=1
savez_compressed("Target File Path Here",labels)

## Concatenate YOLO And Optical Flow

In [ ]:
import glob
import numpy as np
from numpy import savez_compressed
import os

In [ ]:
opticalFlowFilePath = "Optical Flow File Path Here"
opticalFlowFiles = glob.glob(opticalFlowFilePath)

yoloFilePath = "Yolo File Path Here"
yoloFiles = glob.glob(opticalFlowFilePath)

In [ ]:
for i in opticalFlowFiles:
    copyFile1 = i
    videoNames = copyFile1.split("\\")[-1]
    subFiles = glob.glob(i+"\\*")
    for j in subFiles:
        copyFile2 = j
        frameName = copyFile2.split("\\")[-1]
        opticalArray = np.load(j,allow_pickle = True)
        opticalArrayData = opticalArray[opticalArray.files[0]]
        yoloArray = np.load(yoloFilePath+videoNames+"\\"+frameName,allow_pickle = True)
        yoloArrayData = yoloArray[yoloArray.files[0]]
        opticalArrayData = opticalArrayData.flatten()
        yoloArrayData = yoloArrayData.flatten()
        opticalArrayData = opticalArrayData.tolist()
        yoloArrayData = yoloArrayData.tolist()
        opticalArrayData=(opticalArrayData + yoloArrayData)
        try:
            os.mkdir("Target File Path Here"+videoNames)
        except FileExistsError:
            pass
        savez_compressed("Target File Path Here"+videoNames+"\\"+frameName,opticalArrayData)

# Concatenate Frames 5 by 5 And Create Datas Include 5 frames

In [ ]:
import glob
import numpy as np
from numpy import savez_compressed
import os

labelFilePath = "Labels File Path Here"
filesPath = glob.glob(labelFilePath)
filesPath[0]

dataCounter = 1
for video in filesPath:
    for i in range(1,21):
        data = np.load(video+"\\frame"+str(i)+".npz",allow_pickle = True)
        data = data[data.files[0]]
        for j in range(i+1,i+5):
            newData = np.load(video+"\\frame"+str(j)+".npz",allow_pickle = True)
            newData = newData[newData.files[0]]
            data = np.concatenate((data,newData),axis = 0)
        savez_compressed("Target File Path Here"+str(dataCounter),data)
        dataCounter = dataCounter + 1

## Create Final Data Which Includes Data Inside 5 Frames

In [ ]:
import glob
import numpy as np
from numpy import savez_compressed
import os

labelFilePath = "Data Fiel Path Here"

def concatenater(a,b,k,labelFilePath):
    DATA = []
    for i in range(a,b):
        
        data1 = np.load(labelFilePath+"data"+str(i)+".npz",allow_pickle = True)
        data1 = data1[data1.files[0]]
        DATA.append(data1)
    savez_compressed(labelFilePath+"DATA_"+str(k),DATA)
    
for i in range(180):
    concatenater(0+500*i,500+500*i,i,labelFilePath)

## Extract Optical Flow From Videos

In [ ]:
model = VGG16() #for feat extract
model = Model(inputs=model.inputs, outputs=model.layers[-2].output)
def feat_extract(im):
    x = preprocess_input(im)
    features = model.predict(x)
    return features

In [ ]:
def denseOF(videoID):
    cap = cv.VideoCapture(videoLoc)
    ret, frame = cap.read() #get frames from vidya
    framegray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    hsv = np.zeros_like(frame)
    hsv[..., 1] = 244
    
    try:
        while(1):
            ret = cap.grab()
            ret, frame = cap.retrieve() # decode the frame
            if ret: 
                ret2, frame2 = cap.read()
                nxt = cv.cvtColor(frame2, cv.COLOR_BGR2GRAY)
                flow = cv.calcOpticalFlowFarneback(framegray, nxt, None, 0.5, 3, 15, 3, 5, 1.2, 0)
                mag, ang = cv.cartToPolar(flow[..., 0], flow[..., 1])

                hsv[..., 0] = ang * 180 / np.pi / 2
                hsv[..., 2] = cv.normalize(mag, None, 0, 255, cv.NORM_MINMAX)

                rgb = cv.cvtColor(hsv, cv.COLOR_HSV2BGR)
                systemVertical = np.concatenate((frame2, rgb), axis=1)

                magnitudeList = np.asarray(mag)
                orientationList = np.asarray(ang)
                magorientList = mag *ang
                
                orientationList = np.resize(orientationList, (224, 224, 3) )

                transformedImage2 = np.expand_dims(orientationList, axis = 0)  
                
                feats2 = feat_extract(transformedImage2)
            
                savez_compressed(frameLoc2, feats2)
                
                framegray = nxt
                if cv.waitKey(1) & 0xFF == ord('q'):
                    break
            else:
                return
    except:
        cap.release()
        cv.destroyAllWindows()